In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, AvgPool2D, Dropout
from tensorflow.keras.datasets import mnist

In [2]:
def nn_output(elem, max_elem: int=8):
    res = np.zeros(max_elem, dtype=int)
    res[elem] = 1
    return np.array(res)

In [3]:
from augmentation import Image, Augmentation
import cv2
METHODS = ["rotation", "shifting", "blurring"]

SHAPE = (120, 120, 3)

In [4]:
np.random.seed(42)

In [5]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [6]:
model = Sequential()
model.add(Conv2D(6, (5, 5), input_shape = SHAPE, activation = "relu", padding="same"))
model.add(AvgPool2D(pool_size = (2, 2)))
model.add(Conv2D(10, (3, 3), activation = "relu"))
model.add(AvgPool2D(pool_size = (2, 2)))
model.add(Conv2D(17, (5, 5), activation = "relu"))
model.add(AvgPool2D(pool_size = (2, 2)))
model.add(Conv2D(27, (5, 5), activation = "relu"))
model.add(AvgPool2D(pool_size = (2, 2), strides=(2, 2)))
model.add(Conv2D(32, (3, 3), activation = "relu"))
model.add(Flatten())
model.add(Dense(10, kernel_initializer="normal", activation="softmax"))

model.compile(loss="categorical_crossentropy", optimizer="Adam", metrics=["accuracy"])

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 120, 120, 6)       456       
_________________________________________________________________
average_pooling2d (AveragePo (None, 60, 60, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 58, 58, 10)        550       
_________________________________________________________________
average_pooling2d_1 (Average (None, 29, 29, 10)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 25, 25, 17)        4267      
_________________________________________________________________
average_pooling2d_2 (Average (None, 12, 12, 17)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 27)          11502     
__________

In [8]:
for _ in range(2):
    batch_size = 4000
    i_start = 0
    i_finish = batch_size + 1
    for i in range(X_train.shape[0] // batch_size):
        X = []
        y = []
        for elem, label in zip(X_train[i_start: i_finish], y_train[i_start: i_finish]):
            img = Image([0, 0, 0, 0], image_matrix=cv2.merge([255 - elem for _ in range(3)]))
            img = img.shearing(tuple(SHAPE[:2]))
            X.append(img.image_matrix / 255)
            y.append(label)
            aug = Augmentation(img, count=4, methods_list=METHODS)
            aug.create_children()
            for child in aug.children:
                X.append(child.image_matrix / 255)
                y.append(label)
        X = np.array(X)
        y = np.array(y)
        y = np.array(list(map(lambda x: nn_output(x, 10), y)))
        model.fit(X, y, batch_size=1024, epochs=3, verbose=1)
        i_start = i_finish
        i_finish += batch_size

        _X_ = []
        for elem, label in zip(X_test, y_test):
            img = Image([0, 0, 0, 0], image_matrix=cv2.merge([255 - elem for _ in range(3)]))
            img = img.shearing(tuple(SHAPE[:2]))
            _X_.append(img.image_matrix / 255)
        _X_ = np.array(_X_)
        pred = model.predict(_X_)
        print(f"test acc: {np.sum(y_test == pred.argmax(axis=1)) / 10000}")

Epoch 1/2
20005/20005 [==============================] - 112s 6ms/step - loss: 1.8339 - acc: 0.3863
Epoch 2/2
20005/20005 [==============================] - 111s 6ms/step - loss: 0.7661 - acc: 0.7626
test acc: 0.8359
Epoch 1/2
20000/20000 [==============================] - 110s 5ms/step - loss: 0.4744 - acc: 0.8536
Epoch 2/2
20000/20000 [==============================] - 111s 6ms/step - loss: 0.3102 - acc: 0.9028
test acc: 0.9216
Epoch 1/2
20000/20000 [==============================] - 111s 6ms/step - loss: 0.2980 - acc: 0.9104
Epoch 2/2
20000/20000 [==============================] - 111s 6ms/step - loss: 0.2241 - acc: 0.9324
test acc: 0.9401
Epoch 1/2
20000/20000 [==============================] - 110s 6ms/step - loss: 0.2121 - acc: 0.9356
Epoch 2/2
20000/20000 [==============================] - 111s 6ms/step - loss: 0.1533 - acc: 0.9538
test acc: 0.9603
Epoch 1/2
20000/20000 [==============================] - 112s 6ms/step - loss: 0.1414 - acc: 0.9559
Epoch 2/2
20000/20000 [=========

KeyboardInterrupt: 

In [9]:
model.save("my_MNIST.h5")